# Table 4 (Molina et al., submitted)

Table 4. Pearson correlation matrix showing coefficients among the CESM2 (11-member ensemble mean) weather regime 500-hPa geopotential height anomalies (1999-2019) across 10-70°N and 150-40°W. Correlation coefficients for ERA5 weather regimes are shown in parentheses for comparison. Two-tailed statistical significance computed using a 10,000-member bootstrap and indicated in bold-face at the 95% confidence level and with an asterisk at the 99% confidence level.

## Imports

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

from sklearn.decomposition import PCA
from scipy import stats
from sklearn.cluster import KMeans

import som_analysis
import cluster_analysis
import narm_analysis

## function

In [2]:
def get_cold_indx(ds, mo_init=9, mo_end=2):
    """
    Extract indices for cold season.
    Grabbing Sept thru February init, for Oct thru March predictions.
    """
    dt_array = pd.to_datetime(ds['time'])
    return xr.where((dt_array.month >= mo_init) | (dt_array.month <= mo_end),
                    True, False)

## open and preprocess training data

In [3]:
# region for clustering
lat0 = 10
lat1 = 70
lon0 = -150
lon1 = -40

# open era5 data and slice
ds_era5 = narm_analysis.era5_z500(lat0=lat0, lat1=lat1, lon0=lon0, lon1=lon1)

# era5 anomalies
ds_era5_anom = narm_analysis.era5_climo_wrs(
    ds_era5, rolling_days=5, variable='clim')

# restructure era5 array for machine learning training (SONDJFM)
ds_era5_anom = ds_era5_anom[get_cold_indx(
    ds_era5_anom, mo_init=10, mo_end=3), ...]

ds_era5_train = ds_era5_anom.stack(
    flat=('lat', 'lon')).transpose('time', 'flat').values

## pca and kmeans with era5

In [4]:
# create pca object
pca_obj = PCA(12, whiten=True)

# fit pca with era5
pca_obj = pca_obj.fit(ds_era5_train)

# transform era5 data with pca
ds_era5_train = pca_obj.transform(ds_era5_train)

print(f'Variance explained: {pca_obj.explained_variance_ratio_ * 100}')
print(
    f'Cumulative sum of variance explained for EOF1 and EOF2: {np.cumsum(pca_obj.explained_variance_ratio_) * 100}'
)

# train kmeans
k_means = KMeans(n_clusters=4,
                 init='k-means++',
                 n_init=10000,
                 max_iter=300,
                 tol=0.0001,
                 verbose=0,
                 random_state=0).fit(ds_era5_train)

print(f'inertia: {k_means.inertia_}')

Variance explained: [25.95315607 17.65410568 11.94871708  9.0784389   7.98100848  6.14181738
  4.32605934  2.61658689  2.22642929  2.17049559  1.49813958  1.22541708]
Cumulative sum of variance explained for EOF1 and EOF2: [25.95315607 43.60726175 55.55597883 64.63441774 72.61542622 78.7572436
 83.08330294 85.69988983 87.92631912 90.09681471 91.59495429 92.82037136]
inertia: 39379.20538396953


## load hindcast cesm data with lead time bias corrected anomalies (and era5 similarly arranged)

In [5]:
lat0 = 10
lat1 = 70
lon0 = -150
lon1 = -40

# era5 data

z500_era5, z500_era5_dt = som_analysis.open_era5_files(
    variable='z500', return_time=True,
    lat0=lat0, lat1=lat1, lon0=lon0, lon1=lon1,
    leadday0=0, leadday1=42, rolldays=5,)

z500_standard_era5 = z500_era5.stack(
    new=('time', 'lead'), flat=('lat', 'lon')).transpose('new', 'flat')

# cesm data

z500_cesm, z500_cesm_dt = som_analysis.open_cesm_files(
    variable='zg_500', return_time=True,
    lat0=lat0, lat1=lat1, lon0=lon0, lon1=lon1,
    leadday0=0, leadday1=42, rolldays=5,)

z500_standard_cesm = z500_cesm.stack(
    new=('time', 'lead'), flat=('lat', 'lon')).transpose('new', 'flat')

## extract weather regime indices

In [6]:
# grab cluster indices

z500_era5_tmp_1, z500_era5_tmp_2, z500_era5_tmp_3, z500_era5_tmp_4 = cluster_analysis.composite_clusters_indx(
    z500_standard_era5, k_means, pca_obj, use_pca=True)

z500_cesm_tmp_1, z500_cesm_tmp_2, z500_cesm_tmp_3, z500_cesm_tmp_4 = cluster_analysis.composite_clusters_indx(
    z500_standard_cesm, k_means, pca_obj, use_pca=True)

In [7]:
# extract clusters using indices

z500_era5_tmp_01 = z500_standard_era5.unstack('flat').transpose(
    'new', 'lat', 'lon')[z500_era5_tmp_1, :, :]
z500_era5_tmp_02 = z500_standard_era5.unstack('flat').transpose(
    'new', 'lat', 'lon')[z500_era5_tmp_2, :, :]
z500_era5_tmp_03 = z500_standard_era5.unstack('flat').transpose(
    'new', 'lat', 'lon')[z500_era5_tmp_3, :, :]
z500_era5_tmp_04 = z500_standard_era5.unstack('flat').transpose(
    'new', 'lat', 'lon')[z500_era5_tmp_4, :, :]

z500_cesm_tmp_01 = z500_standard_cesm.unstack('flat').transpose(
    'new', 'lat', 'lon')[z500_cesm_tmp_1, :, :]
z500_cesm_tmp_02 = z500_standard_cesm.unstack('flat').transpose(
    'new', 'lat', 'lon')[z500_cesm_tmp_2, :, :]
z500_cesm_tmp_03 = z500_standard_cesm.unstack('flat').transpose(
    'new', 'lat', 'lon')[z500_cesm_tmp_3, :, :]
z500_cesm_tmp_04 = z500_standard_cesm.unstack('flat').transpose(
    'new', 'lat', 'lon')[z500_cesm_tmp_4, :, :]

## bootstrap for table 4 (era5)

In [8]:
all_data0 = z500_standard_era5.unstack('flat').transpose(
    'new', 'lat', 'lon').values
tmp_data1 = z500_era5_tmp_01.values
tmp_data2 = z500_era5_tmp_02.values
tmp_data3 = z500_era5_tmp_03.values
tmp_data4 = z500_era5_tmp_04.values

In [9]:
boot_num_iter_ = 10000

In [10]:
for ind in range(0, boot_num_iter_):

    np.random.seed(ind)
    rand_indx1 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data1.shape[0])]

    np.random.seed(ind + 1)
    rand_indx2 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data2.shape[0])]

    boot_ = stats.pearsonr(
        np.mean(all_data0[rand_indx1, ...], axis=0).flatten(),
        np.mean(all_data0[rand_indx2, ...], axis=0).flatten()
    )[0]

    xr.Dataset(
        data_vars=dict(
            iteration=(["iters"], np.array([boot_])),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/table4_era5_wr12/z500_era5_boot_{ind + 1}.nc')

In [11]:
for ind in range(0, boot_num_iter_):

    np.random.seed(ind)
    rand_indx1 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data1.shape[0])]

    np.random.seed(ind + 1)
    rand_indx3 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data3.shape[0])]

    boot_ = stats.pearsonr(
        np.mean(all_data0[rand_indx1, ...], axis=0).flatten(),
        np.mean(all_data0[rand_indx3, ...], axis=0).flatten()
    )[0]

    xr.Dataset(
        data_vars=dict(
            iteration=(["iters"], np.array([boot_])),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/table4_era5_wr13/z500_era5_boot_{ind + 1}.nc')

In [12]:
for ind in range(0, boot_num_iter_):

    np.random.seed(ind)
    rand_indx1 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data1.shape[0])]

    np.random.seed(ind + 1)
    rand_indx4 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data4.shape[0])]

    boot_ = stats.pearsonr(
        np.mean(all_data0[rand_indx1, ...], axis=0).flatten(),
        np.mean(all_data0[rand_indx4, ...], axis=0).flatten()
    )[0]

    xr.Dataset(
        data_vars=dict(
            iteration=(["iters"], np.array([boot_])),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/table4_era5_wr14/z500_era5_boot_{ind + 1}.nc')

In [13]:
for ind in range(0, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx2 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data2.shape[0])]

    np.random.seed(ind)
    rand_indx3 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data3.shape[0])]

    boot_ = stats.pearsonr(
        np.mean(all_data0[rand_indx2, ...], axis=0).flatten(),
        np.mean(all_data0[rand_indx3, ...], axis=0).flatten()
    )[0]

    xr.Dataset(
        data_vars=dict(
            iteration=(["iters"], np.array([boot_])),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/table4_era5_wr23/z500_era5_boot_{ind + 1}.nc')

In [14]:
for ind in range(0, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx2 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data2.shape[0])]

    np.random.seed(ind)
    rand_indx4 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data4.shape[0])]

    boot_ = stats.pearsonr(
        np.mean(all_data0[rand_indx2, ...], axis=0).flatten(),
        np.mean(all_data0[rand_indx4, ...], axis=0).flatten()
    )[0]

    xr.Dataset(
        data_vars=dict(
            iteration=(["iters"], np.array([boot_])),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/table4_era5_wr24/z500_era5_boot_{ind + 1}.nc')

In [ ]:
for ind in range(0, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx3 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data3.shape[0])]

    np.random.seed(ind)
    rand_indx4 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data4.shape[0])]

    boot_ = stats.pearsonr(
        np.mean(all_data0[rand_indx3, ...], axis=0).flatten(),
        np.mean(all_data0[rand_indx4, ...], axis=0).flatten()
    )[0]

    xr.Dataset(
        data_vars=dict(
            iteration=(["iters"], np.array([boot_])),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/table4_era5_wr34/z500_era5_boot_{ind + 1}.nc')

## bootstrap for table 4 (cesm)

In [ ]:
all_data0 = z500_standard_cesm.unstack('flat').transpose(
    'new', 'lat', 'lon').values
tmp_data1 = z500_cesm_tmp_01.values
tmp_data2 = z500_cesm_tmp_02.values
tmp_data3 = z500_cesm_tmp_03.values
tmp_data4 = z500_cesm_tmp_04.values

In [ ]:
for ind in range(0, boot_num_iter_):

    np.random.seed(ind)
    rand_indx1 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data1.shape[0])]

    np.random.seed(ind + 1)
    rand_indx2 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data2.shape[0])]

    boot_ = stats.pearsonr(
        np.mean(all_data0[rand_indx1, ...], axis=0).flatten(),
        np.mean(all_data0[rand_indx2, ...], axis=0).flatten()
    )[0]

    xr.Dataset(
        data_vars=dict(
            iteration=(["iters"], np.array([boot_])),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/table4_cesm_wr12/z500_cesm_boot_{ind + 1}.nc')

In [ ]:
for ind in range(0, boot_num_iter_):

    np.random.seed(ind)
    rand_indx1 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data1.shape[0])]

    np.random.seed(ind + 1)
    rand_indx3 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data3.shape[0])]

    boot_ = stats.pearsonr(
        np.mean(all_data0[rand_indx1, ...], axis=0).flatten(),
        np.mean(all_data0[rand_indx3, ...], axis=0).flatten()
    )[0]

    xr.Dataset(
        data_vars=dict(
            iteration=(["iters"], np.array([boot_])),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/table4_cesm_wr13/z500_cesm_boot_{ind + 1}.nc')

In [ ]:
for ind in range(0, boot_num_iter_):

    np.random.seed(ind)
    rand_indx1 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data1.shape[0])]

    np.random.seed(ind + 1)
    rand_indx4 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data4.shape[0])]

    boot_ = stats.pearsonr(
        np.mean(all_data0[rand_indx1, ...], axis=0).flatten(),
        np.mean(all_data0[rand_indx4, ...], axis=0).flatten()
    )[0]

    xr.Dataset(
        data_vars=dict(
            iteration=(["iters"], np.array([boot_])),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/table4_cesm_wr14/z500_cesm_boot_{ind + 1}.nc')

In [ ]:
for ind in range(0, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx2 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data2.shape[0])]

    np.random.seed(ind)
    rand_indx3 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data3.shape[0])]

    boot_ = stats.pearsonr(
        np.mean(all_data0[rand_indx2, ...], axis=0).flatten(),
        np.mean(all_data0[rand_indx3, ...], axis=0).flatten()
    )[0]

    xr.Dataset(
        data_vars=dict(
            iteration=(["iters"], np.array([boot_])),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/table4_cesm_wr23/z500_cesm_boot_{ind + 1}.nc')

In [ ]:
for ind in range(0, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx2 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data2.shape[0])]

    np.random.seed(ind)
    rand_indx4 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data4.shape[0])]

    boot_ = stats.pearsonr(
        np.mean(all_data0[rand_indx2, ...], axis=0).flatten(),
        np.mean(all_data0[rand_indx4, ...], axis=0).flatten()
    )[0]

    xr.Dataset(
        data_vars=dict(
            iteration=(["iters"], np.array([boot_])),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/table4_cesm_wr24/z500_cesm_boot_{ind + 1}.nc')

In [ ]:
for ind in range(0, boot_num_iter_):

    np.random.seed(ind + 1)
    rand_indx3 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data3.shape[0])]

    np.random.seed(ind)
    rand_indx4 = [np.random.choice(all_data0.shape[0]) for i in range(
        tmp_data4.shape[0])]

    boot_ = stats.pearsonr(
        np.mean(all_data0[rand_indx3, ...], axis=0).flatten(),
        np.mean(all_data0[rand_indx4, ...], axis=0).flatten()
    )[0]

    xr.Dataset(
        data_vars=dict(
            iteration=(["iters"], np.array([boot_])),
        ),
        attrs=dict(description="For bootstrap confidence intervals."),
    ).to_netcdf(
        f'/glade/scratch/molina/s2s/bootstrap/table4_cesm_wr34/z500_cesm_boot_{ind + 1}.nc')

## compute percentiles

In [22]:
lev_1 = 0.025
lev_2 = 0.975
lev_3 = 0.005
lev_4 = 0.995

In [23]:
tmp_era5_wr12 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/table4_era5_wr12/z500_era5_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_era5_wr13 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/table4_era5_wr13/z500_era5_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_era5_wr14 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/table4_era5_wr14/z500_era5_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_era5_wr23 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/table4_era5_wr23/z500_era5_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_era5_wr24 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/table4_era5_wr24/z500_era5_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_era5_wr34 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/table4_era5_wr34/z500_era5_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

In [24]:
tmp_cesm_wr12 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/table4_cesm_wr12/z500_cesm_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_cesm_wr13 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/table4_cesm_wr13/z500_cesm_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_cesm_wr14 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/table4_cesm_wr14/z500_cesm_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_cesm_wr23 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/table4_cesm_wr23/z500_cesm_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_cesm_wr24 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/table4_cesm_wr24/z500_cesm_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

tmp_cesm_wr34 = xr.open_mfdataset(
    '/glade/scratch/molina/s2s/bootstrap/table4_cesm_wr34/z500_cesm_boot_*.nc',
    combine='nested', concat_dim='iter').chunk(
    dict(iter=-1)).quantile([lev_1, lev_2, lev_3, lev_4], dim='iter', skipna=True)

## data assemble

In [25]:
ds_table4 = xr.Dataset(

    data_vars=dict(

        era5_12=(["quantile", "iters"], tmp_era5_wr12["iteration"].data),
        era5_13=(["quantile", "iters"], tmp_era5_wr13["iteration"].data),
        era5_14=(["quantile", "iters"], tmp_era5_wr14["iteration"].data),
        era5_23=(["quantile", "iters"], tmp_era5_wr23["iteration"].data),
        era5_24=(["quantile", "iters"], tmp_era5_wr24["iteration"].data),
        era5_34=(["quantile", "iters"], tmp_era5_wr34["iteration"].data),

        cesm_12=(["quantile", "iters"], tmp_cesm_wr12["iteration"].data),
        cesm_13=(["quantile", "iters"], tmp_cesm_wr13["iteration"].data),
        cesm_14=(["quantile", "iters"], tmp_cesm_wr14["iteration"].data),
        cesm_23=(["quantile", "iters"], tmp_cesm_wr23["iteration"].data),
        cesm_24=(["quantile", "iters"], tmp_cesm_wr24["iteration"].data),
        cesm_34=(["quantile", "iters"], tmp_cesm_wr34["iteration"].data),

    ),

    coords=dict(
        quantile=(["quantile"], [lev_1, lev_2, lev_3, lev_4]),
        iters=(["iters"], np.arange(0, 1, 1)),
    ),

    attrs=dict(description="For bootstrap confidence intervals."),
)

## save file

In [26]:
ds_table4.to_netcdf('/glade/scratch/molina/s2s/bootstrap/table4_cesm_sig.nc')

# Table 4

In [9]:
ds_table4 = xr.open_dataset(
    '/glade/scratch/molina/s2s/bootstrap/table4_cesm_sig.nc')

In [14]:
print(
    stats.pearsonr(
        z500_era5_tmp_01.mean('new').values.flatten(),
        z500_era5_tmp_02.mean('new').values.flatten()
    )[0],
    "2.5%:", ds_table4['era5_12'].sel(quantile=0.025).values,
    "97.5%:", ds_table4['era5_12'].sel(quantile=0.975).values,
    "0.5%:", ds_table4['era5_12'].sel(quantile=0.005).values,
    "99.5%:", ds_table4['era5_12'].sel(quantile=0.995).values,
)

print(
    stats.pearsonr(
        z500_era5_tmp_01.mean('new').values.flatten(),
        z500_era5_tmp_03.mean('new').values.flatten()
    )[0],
    "2.5%:", ds_table4['era5_13'].sel(quantile=0.025).values,
    "97.5%:", ds_table4['era5_13'].sel(quantile=0.975).values,
    "0.5%:", ds_table4['era5_13'].sel(quantile=0.005).values,
    "99.5%:", ds_table4['era5_13'].sel(quantile=0.995).values,
)

print(
    stats.pearsonr(
        z500_era5_tmp_01.mean('new').values.flatten(),
        z500_era5_tmp_04.mean('new').values.flatten()
    )[0],
    "2.5%:", ds_table4['era5_14'].sel(quantile=0.025).values,
    "97.5%:", ds_table4['era5_14'].sel(quantile=0.975).values,
    "0.5%:", ds_table4['era5_14'].sel(quantile=0.005).values,
    "99.5%:", ds_table4['era5_14'].sel(quantile=0.995).values,
)

print(
    stats.pearsonr(
        z500_era5_tmp_02.mean('new').values.flatten(),
        z500_era5_tmp_03.mean('new').values.flatten()
    )[0],
    "2.5%:", ds_table4['era5_23'].sel(quantile=0.025).values,
    "97.5%:", ds_table4['era5_23'].sel(quantile=0.975).values,
    "0.5%:", ds_table4['era5_23'].sel(quantile=0.005).values,
    "99.5%:", ds_table4['era5_23'].sel(quantile=0.995).values,
)

print(
    stats.pearsonr(
        z500_era5_tmp_02.mean('new').values.flatten(),
        z500_era5_tmp_04.mean('new').values.flatten()
    )[0],
    "2.5%:", ds_table4['era5_24'].sel(quantile=0.025).values,
    "97.5%:", ds_table4['era5_24'].sel(quantile=0.975).values,
    "0.5%:", ds_table4['era5_24'].sel(quantile=0.005).values,
    "99.5%:", ds_table4['era5_24'].sel(quantile=0.995).values,
)

print(
    stats.pearsonr(
        z500_era5_tmp_03.mean('new').values.flatten(),
        z500_era5_tmp_04.mean('new').values.flatten()
    )[0],
    "2.5%:", ds_table4['era5_34'].sel(quantile=0.025).values,
    "97.5%:", ds_table4['era5_34'].sel(quantile=0.975).values,
    "0.5%:", ds_table4['era5_34'].sel(quantile=0.005).values,
    "99.5%:", ds_table4['era5_34'].sel(quantile=0.995).values,
)

-0.3766447998573811 2.5%: [-0.01286973] 97.5%: [0.83883486] 0.5%: [-0.17630252] 99.5%: [0.88093041]
-0.5931004999055469 2.5%: [0.00348226] 97.5%: [0.84106366] 0.5%: [-0.12921935] 99.5%: [0.88685887]
0.2120332977004566 2.5%: [0.04075366] 97.5%: [0.84885605] 0.5%: [-0.17073519] 99.5%: [0.88546372]
0.001096920448808159 2.5%: [0.0478954] 97.5%: [0.85153731] 0.5%: [-0.13087273] 99.5%: [0.89396982]
-0.680488619493687 2.5%: [0.10468421] 97.5%: [0.85287131] 0.5%: [-0.14165574] 99.5%: [0.8889681]
-0.5267873364990872 2.5%: [0.12243644] 97.5%: [0.86534154] 0.5%: [-0.02626205] 99.5%: [0.8992843]


# Table 4

In [15]:
print(
    stats.pearsonr(
        z500_cesm_tmp_01.mean('new').values.flatten(),
        z500_cesm_tmp_02.mean('new').values.flatten()
    )[0],
    "2.5%:", ds_table4['cesm_12'].sel(quantile=0.025).values,
    "97.5%:", ds_table4['cesm_12'].sel(quantile=0.975).values,
    "0.5%:", ds_table4['cesm_12'].sel(quantile=0.005).values,
    "99.5%:", ds_table4['cesm_12'].sel(quantile=0.995).values,
)

print(
    stats.pearsonr(
        z500_cesm_tmp_01.mean('new').values.flatten(),
        z500_cesm_tmp_03.mean('new').values.flatten()
    )[0],
    "2.5%:", ds_table4['cesm_13'].sel(quantile=0.025).values,
    "97.5%:", ds_table4['cesm_13'].sel(quantile=0.975).values,
    "0.5%:", ds_table4['cesm_13'].sel(quantile=0.005).values,
    "99.5%:", ds_table4['cesm_13'].sel(quantile=0.995).values,
)

print(
    stats.pearsonr(
        z500_cesm_tmp_01.mean('new').values.flatten(),
        z500_cesm_tmp_04.mean('new').values.flatten()
    )[0],
    "2.5%:", ds_table4['cesm_14'].sel(quantile=0.025).values,
    "97.5%:", ds_table4['cesm_14'].sel(quantile=0.975).values,
    "0.5%:", ds_table4['cesm_14'].sel(quantile=0.005).values,
    "99.5%:", ds_table4['cesm_14'].sel(quantile=0.995).values,
)

print(
    stats.pearsonr(
        z500_cesm_tmp_02.mean('new').values.flatten(),
        z500_cesm_tmp_03.mean('new').values.flatten()
    )[0],
    "2.5%:", ds_table4['cesm_23'].sel(quantile=0.025).values,
    "97.5%:", ds_table4['cesm_23'].sel(quantile=0.975).values,
    "0.5%:", ds_table4['cesm_23'].sel(quantile=0.005).values,
    "99.5%:", ds_table4['cesm_23'].sel(quantile=0.995).values,
)

print(
    stats.pearsonr(
        z500_cesm_tmp_02.mean('new').values.flatten(),
        z500_cesm_tmp_04.mean('new').values.flatten()
    )[0],
    "2.5%:", ds_table4['cesm_24'].sel(quantile=0.025).values,
    "97.5%:", ds_table4['cesm_24'].sel(quantile=0.975).values,
    "0.5%:", ds_table4['cesm_24'].sel(quantile=0.005).values,
    "99.5%:", ds_table4['cesm_24'].sel(quantile=0.995).values,
)

print(
    stats.pearsonr(
        z500_cesm_tmp_03.mean('new').values.flatten(),
        z500_cesm_tmp_04.mean('new').values.flatten()
    )[0],
    "2.5%:", ds_table4['cesm_34'].sel(quantile=0.025).values,
    "97.5%:", ds_table4['cesm_34'].sel(quantile=0.975).values,
    "0.5%:", ds_table4['cesm_34'].sel(quantile=0.005).values,
    "99.5%:", ds_table4['cesm_34'].sel(quantile=0.995).values,
)

-0.44678411692678993 2.5%: [0.47183606] 97.5%: [0.9387222] 0.5%: [0.34460538] 99.5%: [0.95535471]
-0.5827193144694967 2.5%: [0.55327057] 97.5%: [0.95069667] 0.5%: [0.43270438] 99.5%: [0.96635406]
0.3703679912538275 2.5%: [0.56172097] 97.5%: [0.95287856] 0.5%: [0.44123668] 99.5%: [0.96412097]
0.000705497963714943 2.5%: [0.62019315] 97.5%: [0.9524187] 0.5%: [0.50647252] 99.5%: [0.96010411]
-0.7080287639512566 2.5%: [0.6187312] 97.5%: [0.9582789] 0.5%: [0.52534966] 99.5%: [0.96854267]
-0.5454904719272486 2.5%: [0.68430026] 97.5%: [0.96684872] 0.5%: [0.6106101] 99.5%: [0.97458403]
